In [1]:
import time
import numpy as np

In [2]:
from shared.helpers import build_set

# Gets the histogram sets with labels
print('Loading datasets...')
load_time_start = time.perf_counter()

labels = np.array([0,1])
y_train, x_train = build_set('train', binary=True, use_moments=True, with_extensions=True)
y_test, x_test = build_set('test', binary=True, use_moments=True, with_extensions=True)

print(f'Loaded datasets. Time taken: {time.perf_counter() - load_time_start} s')

Loading datasets...
Loading train dataset...
Loaded 17334 images (with extensions) in 9.4160596 s
Getting image descriptors...
Loaded descriptors for label 0:  (6858, 2)
Loaded descriptors for label 1:  (3138, 2)
Loaded descriptors for label 2:  (4548, 2)
Loaded descriptors for label 3:  (2271, 2)
Loaded descriptors for label 4:  (519, 2)
Total image loading time: 21.7948013 s...
Loading test dataset...
Loaded 4968 images (with extensions) in 2.684984700000001 s
Getting image descriptors...
Loaded descriptors for label 0:  (1917, 2)
Loaded descriptors for label 1:  (888, 2)
Loaded descriptors for label 2:  (1341, 2)
Loaded descriptors for label 3:  (669, 2)
Loaded descriptors for label 4:  (153, 2)
Total image loading time: 3.7580674000000016 s...
Loaded datasets. Time taken: 25.757414 s


In [3]:
import importlib

helpers = importlib.import_module('shared.helpers')
importlib.reload(helpers)

from shared.helpers import scale_dataset

print('unscaled: ', x_train[:2])

x_scaled = x_train
x_test_scaled = x_test

x_scaled = scale_dataset(x_scaled)
x_test_scaled = scale_dataset(x_test_scaled)

print('scaled: ', x_scaled[:2])

unscaled:  [[ -3.1523345  -7.2572093 -11.15301   -12.524306  -24.662037  -16.436357
  -24.42609  ]
 [ -3.1742635  -7.6360016 -11.778712  -12.834935  -25.586607  -17.294075
  -25.171728 ]]
Scaling and fitting dataset...
Scaling and fitting dataset...
scaled:  [[-0.23317589  0.54865736  0.15126559 -0.11027179 -0.06145787  0.07264346
   0.11679873]
 [-0.5273372  -0.59333515 -1.1655784  -0.5280247  -0.7270186  -0.75469023
  -0.46720853]]


In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_neighbors': range(1, 120),
    'weights': ['uniform', 'distance']
}

knn = KNeighborsClassifier(n_jobs=-1)
clf = GridSearchCV(knn, params, cv=5)
clf.fit(x_scaled, y_train)

clf = clf.best_estimator_

KeyboardInterrupt: 

In [5]:
from sklearn.metrics import confusion_matrix, classification_report

y_predict=clf.predict(x_test_scaled)
accuracy = clf.score(x_test_scaled, y_test)

print(f"Accuracy: {accuracy * 100}%")
print("\nClassification report:")
print(classification_report(y_test, y_predict))

print("\nConfusion matrix:")
print(f"Labels: {labels}\n")
print(confusion_matrix(y_test, y_predict, labels=labels))

Accuracy: 56.28019323671497%

Classification report:
              precision    recall  f1-score   support

           0       0.60      0.68      0.64      2805
           1       0.50      0.42      0.45      2163

    accuracy                           0.56      4968
   macro avg       0.55      0.55      0.54      4968
weighted avg       0.56      0.56      0.56      4968


Confusion matrix:
Labels: [0 1]

[[1898  907]
 [1265  898]]


In [7]:
import os
import joblib

import shared.constants as constants

if os.path.exists(constants.MODELS_DIR):
    joblib.dump(clf, 'shallow_bin.sav')
else:
    print('Cannot save trained svm model to shallow_binary.sav')